In [2]:
import json
import pandas as pd
import requests
from pprint import pprint
from pandas.io.json import json_normalize  

In [24]:
#Import CSVs
aqi2019_df = pd.read_csv("assets/sample_data/daily_aqi_by_cbsa_2019.csv")
aqi2020_df = pd.read_csv("assets/sample_data/daily_aqi_by_cbsa_2020.csv")

#Append the CSVs to one big dataframe
aqi_df = aqi2019_df.append(aqi2020_df)
aqi_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,"Aberdeen, SD",10100,2019-01-03,4,Good,PM10,46-013-0003,1
1,"Aberdeen, SD",10100,2019-01-06,31,Good,PM2.5,46-013-0003,1
2,"Aberdeen, SD",10100,2019-01-09,9,Good,PM10,46-013-0003,1
3,"Aberdeen, SD",10100,2019-01-12,43,Good,PM2.5,46-013-0003,1
4,"Aberdeen, SD",10100,2019-01-15,28,Good,PM2.5,46-013-0003,1


In [25]:
#Rename cities with duplicate names to avoid merging the wrong data
aqi_df = aqi_df.replace({"Columbia, MO": "ColumbiaMO", "Salem, OH": "SalemOH", "Columbus, GA-AL" : "ColumbusGA", "Columbus, IN" : "ColumbusIN",
                                "Jackson, TN": "JacksonTN", "Jackson, WY-ID": "JacksonWY", "Miami, OK": "MiamiOK", "Salem, OH": "SalemOH",
                                "Springfield, MA": "SpringfieldMA", "Springfield, MO" : "SpringfieldMO", "Springfield, OH": "SpringfieldOH"})



In [26]:
#Split CBSA into City and State
aqi_df[['City','State']] = aqi_df.CBSA.str.split(",",expand=True)

aqi_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,City,State
0,"Aberdeen, SD",10100,2019-01-03,4,Good,PM10,46-013-0003,1,Aberdeen,SD
1,"Aberdeen, SD",10100,2019-01-06,31,Good,PM2.5,46-013-0003,1,Aberdeen,SD
2,"Aberdeen, SD",10100,2019-01-09,9,Good,PM10,46-013-0003,1,Aberdeen,SD
3,"Aberdeen, SD",10100,2019-01-12,43,Good,PM2.5,46-013-0003,1,Aberdeen,SD
4,"Aberdeen, SD",10100,2019-01-15,28,Good,PM2.5,46-013-0003,1,Aberdeen,SD


In [27]:
#Rename City metropolitan areas to one main city for merging
aqi_df = aqi_df.replace({"Atlanta-Sandy Springs-Roswell":"Atlanta","Austin-Round Rock":"Austin","Baltimore-Columbia-Towson":"Baltimore",
                        "Boise City":"Boise","Boston-Cambridge-Newton":"Boston","New York-Newark-Jersey City":"New York City",                                                  "Charlotte-Concord-Gastonia":"Charlotte","Chicago-Naperville-Elgin":"Chicago","Dallas-Fort Worth-Arlington":"Dallas",
                        "Denver-Aurora-Lakewood":"Denver","Detroit-Warren-Dearborn":"Detroit","Hartford-West Hartford-East Hartford":"Hartford","Urban                          Honolulu":"Honolulu", "Houston-The Woodlands-Sugar Land":"Houston","Indianapolis-Carmel-Anderson":"Indianapolis","Las                                   Vegas-Henderson-Paradise":"Las Vegas","Little Rock-North Little Rock-Conway":"Little Rock","Los Angeles-Long Beach-Anaheim":"Los Angeles","Miami-Fort Lauderdale-West Palm Beach":"Miami","Milwaukee-Waukesha-West Allis":"Milwaukee",                     
                        "Nashville-Davidson--Murfreesboro--Franklin":"Nashville","Omaha-Council Bluffs":"Omaha","Philadelphia-Camden-Wilmington":
                        "Philadelphia","Phoenix-Mesa-Scottsdale":"Phoenix","Portland-Vancouver-Hillsboro":"Portland","Providence-Warwick":"Providence",
                        "Sacramento--Roseville--Arden-Arcade":"Sacramento","Minneapolis-St. Paul-Bloomington":"Saint Paul","San Antonio-New Braunfels":
                        "San Antonio","San Diego-Carlsbad":"San Diego","San Francisco-Oakland-Hayward":"San Francisco","San Jose-Sunnyvale-Santa Clara":
                        "San Jose","Seattle-Tacoma-Bellevue":"Seattle","Washington-Arlington-Alexandria":"Washington, D.C."})

In [28]:
#Replace state values of multiple states with one state
aqi_df = aqi_df.replace({" MA-NH":"MA"," IL-IN-WI":"IN"," NC-SC":"NC"," TN-MS-AR":"TN"," MN-WI":"MN"," NY-NJ-PA":"NY"," NE-IA":"NE"," PA-NJ-DE-MD":"PA",                                    " OR-WA":"OR"," RI-MA":"RI"," DC-VA-MD-WV":"DC"})

In [29]:
aqi_df["State"].value_counts()

 CA          13482
 TX          10312
 FL           8826
 PA           8652
 WA           7471
             ...  
 KY-IL         369
 PA-NJ         365
 WI-MN         365
 IL-MO         241
 IA-IL-MO      151
Name: State, Length: 94, dtype: int64

In [9]:
# URL for JSON with cities and geo data
url = "https://aqicn.org/data-platform/covid19/airquality-covid19-cities.json"

In [24]:
#Read JSON into a dataframe
df = pd.read_json(url)

df.head()

,csvsize,data,generated
0,46726470,"{'Place': {'geo': [38.53575, 68.77905], 'featu...",2020-05-28T20:54:02+01:00
1,46726470,"{'Place': {'geo': [24.45118, 54.39696], 'featu...",2020-05-28T20:54:02+01:00
2,46726470,"{'Place': {'geo': [25.07725, 55.30927], 'featu...",2020-05-28T20:54:02+01:00
3,46726470,"{'Place': {'geo': [60.29414, 25.04099], 'featu...",2020-05-28T20:54:02+01:00
4,46726470,"{'Place': {'geo': [60.45148, 22.26869], 'featu...",2020-05-28T20:54:02+01:00


In [10]:
# Uses requests.get() to load JSON into text
response = json.loads(requests.get(url).text)

#Flattens the nested JSON of the 'data' column into its own DataFrame to get the Places geo data
geo_df = pd.io.json.json_normalize(response['data'])
#pprint(response)
geo_df.head()

,Sources,Stations,Place.geo,Place.feature,Place.name,Place.country,Place.pop
0,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Jemeppe-sur-meuse, Belgium'}, {'Nam...","[50.63373, 5.56749]",PPL,Liège,BE,182597
1,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Sint-agatha-berchem, Belgium'}, {'N...","[50.85045, 4.34878]",PPLC,Brussels,BE,1019022
2,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Charleroi, Belgium'}, {'Name': 'Mar...","[50.41136, 4.44448]",PPL,Charleroi,BE,200132
3,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Sint-kruiswinkel, Belgium'}, {'Name...","[51.05, 3.71667]",PPL,Gent,BE,231493
4,[{'name': 'Citizen Weather Observer Program (C...,"[{'Name': 'Vezin, Belgium'}, {'Name': 'Namur, ...","[50.4669, 4.86746]",PPLA,Namur,BE,106284


In [11]:
#Puts the geo data and name into a dataframe
places_df = geo_df[['Place.geo','Place.name','Place.country','Place.pop']]

In [9]:
places_df.head()

,Place.geo,Place.name,Place.country
0,"[38.71667, -9.13333]",Lisbon,PT
1,"[32.66568, -16.92547]",Funchal,PT
2,"[55.15402, 61.42915]",Chelyabinsk,RU
3,"[56.32867, 44.00205]",Nizhniy Novgorod,RU
4,"[59.93863, 30.31413]",Saint Petersburg,RU


In [12]:
#Rename columns for merging purposes
places_df = places_df.rename(columns = {'Place.geo':'Geo','Place.name':'City', 'Place.country':'Country', "Place.pop":"Population"})
places_df.head()

,Geo,City,Country
0,"[50.63373, 5.56749]",Liège,BE
1,"[50.85045, 4.34878]",Brussels,BE
2,"[50.41136, 4.44448]",Charleroi,BE
3,"[51.05, 3.71667]",Gent,BE
4,"[50.4669, 4.86746]",Namur,BE


In [13]:
us_df = places_df.loc[places_df["Country"] == "US"]
print(us_df["City"].value_counts())
us_df["City"].isin(aqi_df["City"]).value_counts()

Phoenix             1
Fort Worth          1
Queens              1
Las Vegas           1
Hartford            1
Miami               1
Chicago             1
San Diego           1
Manhattan           1
Atlanta             1
Milwaukee           1
Tallahassee         1
Seattle             1
El Paso             1
Saint Paul          1
Springfield         1
Madison             1
Nashville           1
San Francisco       1
Austin              1
Boise               1
Honolulu            1
San Antonio         1
Indianapolis        1
Richmond            1
Staten Island       1
Washington, D.C.    1
Providence          1
Boston              1
Oakland             1
The Bronx           1
Detroit             1
Jacksonville        1
Sacramento          1
Columbia            1
Brooklyn            1
Tucson              1
Los Angeles         1
Philadelphia        1
Omaha               1
Columbus            1
Little Rock         1
Charlotte           1
Albuquerque         1
Jackson             1
Memphis   

True     48
False     9
Name: City, dtype: int64

In [14]:
#Replace Manhattan with New York City for the merge
us_df = us_df.replace({"Manhattan":"New York City"})

In [21]:
us_df.to_csv("us_locations.csv")

In [30]:
#Inner merge
new_df = aqi_df.merge(us_df,on="City",how="inner")

new_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,City,State,Geo,Country
0,"Albuquerque, NM",10740,2019-01-01,42,Good,PM10,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US
1,"Albuquerque, NM",10740,2019-01-02,40,Good,PM2.5,35-001-1012,8,Albuquerque,NM,"[35.08449, -106.65114]",US
2,"Albuquerque, NM",10740,2019-01-03,55,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US
3,"Albuquerque, NM",10740,2019-01-04,57,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US
4,"Albuquerque, NM",10740,2019-01-05,66,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US


In [31]:

new_df["State"].value_counts()

 CA    2433
 TX    2013
 FL    1276
 AZ     913
 WI     730
NC      487
 UT     487
NE      486
 CO     468
OR      461
 OK     457
 SC     457
 AR     457
NY      457
 OH     457
 MS     457
 NC     457
 IN     456
 GA     456
IN      456
TN      456
DC      456
 VA     456
 MD     455
PA      453
RI      429
MA      426
 IL     418
 WA     397
 NM     365
 CT     365
MN      365
 MI     365
 TN     365
 ID     365
 OR     356
Name: State, dtype: int64

In [73]:
#Store data to CSV for verification
new_df.to_csv("sample_data.csv")

In [32]:
#Split up the location data into latitude and longitude
loc_df = pd.DataFrame(new_df['Geo'].to_list(),index=None, columns =['Lat','Lng'])

loc_df

,Lat,Lng
0,35.08449,-106.65114
1,35.08449,-106.65114
2,35.08449,-106.65114
3,35.08449,-106.65114
4,35.08449,-106.65114
...,...,...
20808,38.89511,-77.03637
20809,38.89511,-77.03637
20810,38.89511,-77.03637
20811,38.89511,-77.03637


In [33]:
#Join the divided lat/lng df to the main dataframe
data_df = new_df.join(loc_df)

data_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,City,State,Geo,Country,Lat,Lng
0,"Albuquerque, NM",10740,2019-01-01,42,Good,PM10,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US,35.08449,-106.65114
1,"Albuquerque, NM",10740,2019-01-02,40,Good,PM2.5,35-001-1012,8,Albuquerque,NM,"[35.08449, -106.65114]",US,35.08449,-106.65114
2,"Albuquerque, NM",10740,2019-01-03,55,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US,35.08449,-106.65114
3,"Albuquerque, NM",10740,2019-01-04,57,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US,35.08449,-106.65114
4,"Albuquerque, NM",10740,2019-01-05,66,Moderate,PM2.5,35-001-0029,8,Albuquerque,NM,"[35.08449, -106.65114]",US,35.08449,-106.65114


In [34]:
#Clean up the dataframe and keep only the desired fields
data_df = data_df[["Date","City","State","Lat","Lng","AQI","Category","Defining Parameter"]]

data_df

,Date,City,State,Lat,Lng,AQI,Category,Defining Parameter
0,2019-01-01,Albuquerque,NM,35.08449,-106.65114,42,Good,PM10
1,2019-01-02,Albuquerque,NM,35.08449,-106.65114,40,Good,PM2.5
2,2019-01-03,Albuquerque,NM,35.08449,-106.65114,55,Moderate,PM2.5
3,2019-01-04,Albuquerque,NM,35.08449,-106.65114,57,Moderate,PM2.5
4,2019-01-05,Albuquerque,NM,35.08449,-106.65114,66,Moderate,PM2.5
...,...,...,...,...,...,...,...,...
20808,2020-03-27,"Washington, D.C.",DC,38.89511,-77.03637,44,Good,Ozone
20809,2020-03-28,"Washington, D.C.",DC,38.89511,-77.03637,40,Good,Ozone
20810,2020-03-29,"Washington, D.C.",DC,38.89511,-77.03637,39,Good,PM2.5
20811,2020-03-30,"Washington, D.C.",DC,38.89511,-77.03637,44,Good,Ozone


In [35]:
data_df.to_csv("sample_data.csv", index=False)

In [36]:
#Get value counts of the cities to see where data discrepancies are
data_df["City"].value_counts()

Charlotte           487
Salt Lake City      487
Omaha               486
Fresno              486
Denver              468
Portland            461
Oklahoma City       457
Little Rock         457
Columbia            457
Jacksonville        457
Jackson             457
Raleigh             457
Columbus            457
New York City       457
Phoenix             457
Memphis             456
Atlanta             456
Tucson              456
Indianapolis        456
Washington, D.C.    456
Richmond            456
San Antonio         456
Chicago             456
Baltimore           455
Miami               454
Philadelphia        453
Providence          429
Boston              426
San Jose            425
Sacramento          425
Springfield         418
Dallas              397
El Paso             397
Seattle             397
Austin              397
San Diego           367
Houston             366
Milwaukee           365
Detroit             365
Madison             365
Boise               365
Hartford        

In [3]:
missing2020_df = pd.read_csv("aqidaily2020.csv")
data_df = pd.read_csv("sample_data.csv")

In [9]:
print(missing2020_df.dtypes)
missing2020_df.head()

Date                   object
CITY                   object
State                  object
Lat                   float64
Lng                   float64
AQI                     int64
Category              float64
Defining Parameter     object
Source                 object
CO                     object
Ozone                  object
SO2                    object
PM10                   object
PM25                   object
NO2                    object
dtype: object


,Date,CITY,State,Lat,Lng,AQI,Category,Defining Parameter,Source,CO,Ozone,SO2,PM10,PM25,NO2
0,1/1/2020,Albuquerque,NM,35.08449,-106.65114,54,NaN,PM2.5,AirNow,42,54,NaN,NaN,NaN,NaN
1,1/2/2020,Albuquerque,NM,35.08449,-106.65114,39,NaN,Ozone,AirNow,39,30,NaN,NaN,NaN,NaN
2,1/3/2020,Albuquerque,NM,35.08449,-106.65114,38,NaN,Ozone,AirNow,38,37,NaN,NaN,NaN,NaN
3,1/4/2020,Albuquerque,NM,35.08449,-106.65114,50,NaN,PM2.5,AirNow,36,50,NaN,NaN,NaN,NaN
4,1/5/2020,Albuquerque,NM,35.08449,-106.65114,63,NaN,PM2.5,AirNow,37,63,NaN,NaN,NaN,NaN


In [18]:
missing2020_df.loc[missing2020_df["AQI"] <= 50, 'Category'] = "Good"
missing2020_df.loc[(missing2020_df["AQI"] > 50) & (missing2020_df["AQI"] <= 100), 'Category'] = "Moderate"
missing2020_df.loc[(missing2020_df["AQI"] > 100) & (missing2020_df["AQI"] <= 150), 'Category'] = "Unhealthy for Sensitive Groups"
missing2020_df.loc[(missing2020_df["AQI"] > 150) & (missing2020_df["AQI"] <= 200), 'Category'] = "Unhealthy"
missing2020_df.loc[(missing2020_df["AQI"] > 250) & (missing2020_df["AQI"] <= 300), 'Category'] = "Very Unhealthy"
missing2020_df.loc[missing2020_df["AQI"] > 300, 'Category'] = "Hazardous"

In [19]:
missing2020_df.head()

,Date,CITY,State,Lat,Lng,AQI,Category,Defining Parameter,Source,CO,Ozone,SO2,PM10,PM25,NO2
0,1/1/2020,Albuquerque,NM,35.08449,-106.65114,54,Moderate,PM2.5,AirNow,42,54,NaN,NaN,NaN,NaN
1,1/2/2020,Albuquerque,NM,35.08449,-106.65114,39,Good,Ozone,AirNow,39,30,NaN,NaN,NaN,NaN
2,1/3/2020,Albuquerque,NM,35.08449,-106.65114,38,Good,Ozone,AirNow,38,37,NaN,NaN,NaN,NaN
3,1/4/2020,Albuquerque,NM,35.08449,-106.65114,50,Good,PM2.5,AirNow,36,50,NaN,NaN,NaN,NaN
4,1/5/2020,Albuquerque,NM,35.08449,-106.65114,63,Moderate,PM2.5,AirNow,37,63,NaN,NaN,NaN,NaN


In [22]:
missing2020_df = missing2020_df.rename(columns = {"CITY":"City","Defining Parameter":"Defining_Parameter"})
data_df = data_df.rename({"Defining Parameter":"Defining_Parameter"})

In [23]:
missing2020_df = missing2020_df[["Date","City","State","Lat","Lng","AQI","Category","Defining_Parameter"]]

In [24]:
data_df = data_df.append(missing2020_df)

In [25]:
data_df["City"].value_counts()

Charlotte           637
Salt Lake City      637
Omaha               636
Fresno              636
Denver              618
Portland            611
Phoenix             608
Columbia            607
Jackson             607
Columbus            607
New York City       607
Jacksonville        607
Raleigh             607
Oklahoma City       607
Atlanta             606
Chicago             606
Memphis             606
Tucson              606
Indianapolis        606
San Antonio         606
Little Rock         606
Richmond            606
Washington, D.C.    606
Baltimore           604
Miami               604
Philadelphia        603
Providence          579
Boston              576
San Jose            575
Sacramento          575
Springfield         566
Seattle             547
Dallas              545
Austin              545
El Paso             544
San Diego           517
Saint Paul          516
Boise               515
San Francisco       515
Los Angeles         515
Hartford            515
Nashville       